# IsCyclic

In [1]:
import torch
from torch_geometric.loader import DataLoader

from GNN import GCN_IsCyclic
from gen_dataset import create_isCyclic_dataset

## Data

In [2]:
dataset, data_objs = create_isCyclic_dataset(saved=True)
print(dataset[0].x)
print(dataset[0].edge_index)
print(dataset[0].y)

# dataset = dataset.shuffle()

Dataset size:  951
tensor([[1.],
        [1.],
        [1.]])
tensor([[0, 0, 1, 1, 2, 2],
        [1, 2, 0, 2, 0, 1]])
tensor(1., dtype=torch.float64)


In [3]:
train_dataset = dataset[:int(0.8 * len(dataset))]
test_dataset = dataset[int(0.8 * len(dataset)):]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[2260, 1], edge_index=[2, 7354], y=[64], batch=[2260], ptr=[65])


## Model

In [6]:
model = GCN_IsCyclic(
    in_features=dataset.num_node_features,
    h_features=128,
    n_classes=2
)

In [7]:
print(model)

GCN_IsCyclic(
  (conv1): GraphConvolution (1 -> 128)
  (conv2): GraphConvolution (128 -> 128)
  (conv3): GraphConvolution (128 -> 128)
  (dense1): Linear(in_features=128, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=8, bias=True)
  (dense3): Linear(in_features=8, out_features=1, bias=True)
)


## Train

In [8]:
for data in train_loader:
    print(data)
    break

DataBatch(x=[1887, 1], edge_index=[2, 10484], y=[64], batch=[1887], ptr=[65])


In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch).squeeze()  # Perform a single forward pass.
        loss = criterion(out, data.y.float())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch).squeeze()
        pred = out.round() # Use the class with highest probability.
        correct += int((pred == data.y.long()).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [21]:
best_test_acc = 0.0
for epoch in range(1, 101):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    if test_acc >= best_test_acc:
        best_test_acc = test_acc
        best_model_params = model.state_dict()
        print("Checkpoint saved!")

Epoch: 001, Train Acc: 0.6987, Test Acc: 0.2565
Checkpoint saved!
Epoch: 002, Train Acc: 0.9079, Test Acc: 1.0000
Checkpoint saved!
Epoch: 003, Train Acc: 0.9408, Test Acc: 0.9895
Epoch: 004, Train Acc: 0.9513, Test Acc: 0.9895
Epoch: 005, Train Acc: 0.9461, Test Acc: 0.9843
Epoch: 006, Train Acc: 0.9158, Test Acc: 1.0000
Checkpoint saved!
Epoch: 007, Train Acc: 0.9605, Test Acc: 1.0000
Checkpoint saved!
Epoch: 008, Train Acc: 0.9224, Test Acc: 1.0000
Checkpoint saved!
Epoch: 009, Train Acc: 0.9553, Test Acc: 0.9895
Epoch: 010, Train Acc: 0.9421, Test Acc: 0.9843
Epoch: 011, Train Acc: 0.9618, Test Acc: 0.9895
Epoch: 012, Train Acc: 0.9987, Test Acc: 0.9948
Epoch: 013, Train Acc: 0.9447, Test Acc: 0.9843


KeyboardInterrupt: 